In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/archive.zip" -d "data"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
import os
import glob
from PIL import Image
import math
from skimage import io
from skimage.transform import resize
import cv2
from sklearn.model_selection import train_test_split
import torchvision.models as models

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 24
LEARNING_RATE = 0.001
NUM_EPOCHS = 25
GAMMA = 0.1

# CPU or GPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [ ]:
dictionary = {
    0: 'Apple___Apple_scab',
    1: 'Apple___Black_rot',
    2: 'Apple___Cedar_apple_rust',
    3: 'Apple___healthy',
    4: 'Blueberry___healthy',
    5: 'Cherry_(including_sour)___healthy',
    6: 'Cherry_(including_sour)___Powdery_mildew',
    7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    8: 'Corn_(maize)___Common_rust_',
    9: 'Corn_(maize)___healthy',
    10: 'Corn_(maize)___Northern_Leaf_Blight',
    11: 'Grape___Black_rot',
    12: 'Grape___Esca_(Black_Measles)',
    13: 'Grape___healthy',
    14: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    15: 'Orange___Haunglongbing_(Citrus_greening)',
    16: 'Peach___Bacterial_spot',
    17: 'Peach___healthy',
    18: 'Pepper,_bell___Bacterial_spot',
    19: 'Pepper,_bell___healthy',
    20: 'Potato___Early_blight',
    21: 'Potato___healthy',
    22: 'Potato___Late_blight',
    23: 'Raspberry___healthy',
    24: 'Soybean___healthy',
    25: 'Squash___Powdery_mildew',
    26: 'Strawberry___healthy',
    27: 'Strawberry___Leaf_scorch',
    28: 'Tomato___Bacterial_spot',
    29: 'Tomato___Early_blight',
    30: 'Tomato___healthy',
    31: 'Tomato___Late_blight',
    32: 'Tomato___Leaf_Mold',
    33: 'Tomato___Septoria_leaf_spot',
    34: 'Tomato___Spider_mites Two-spotted_spider_mite',
    35: 'Tomato___Target_Spot',
    36: 'Tomato___Tomato_mosaic_virus',
    37: 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
}

In [ ]:
x_train = []
y_train = []

def label_folder(folder_path, label):
    images = []
    labels = []
    i = 0
    for img_path in os.listdir(folder_path):
        if i < 400 :
            i = i + 1
            if img_path.endswith('.jpg') or img_path.endswith('.png') or img_path.endswith('.jpeg') or img_path.endswith('.JPG'):
                img = io.imread(str(folder_path)+'/'+str(img_path))
                images.append(img)
                labels.append(label)
            else: 
                print(img_path)
        else:
            break

    x_train.extend(images)
    y_train.extend(labels)
    
## load data
for i in range (38):
    folder_path = os.path.join('/content/data/plantvillage dataset/color/', dictionary[i])
    label_folder(folder_path , i)




In [ ]:
image_size = (224, 224, 3)
class ToTensor:
    # Convert ndarrays to Tensors
    def __call__(self, sample):
        images, targets = sample
        images = resize(images, image_size).reshape(3, 224, 224)
        return torch.from_numpy(np.asarray(images)), torch.from_numpy(np.asarray(targets))

In [ ]:
# transformers, read dataset and split, loader

class DiseaseDataset(Dataset):

    def __init__(self,imgaes, labels, transform= None):
        self.n_samples = len(labels)
        self.x_data = imgaes # size [n_samples, n_features]
        self.y_data = labels # size [n_samples, 1]
        self.transform = transform
    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        sample = self.transform((self.x_data[index], self.y_data[index]))
        return sample

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [ ]:
train_transform = transforms.Compose([ToTensor(), transforms.Normalize(0.5, 0.5)])
test_transform = transforms.Compose([ToTensor(), transforms.Normalize(0.5, 0.5)])

train_images, test_images, train_labels, test_labels = train_test_split(x_train, y_train, test_size=0.2)

train_dataset = DiseaseDataset(train_images, train_labels, transform = ToTensor())
test_dataset = DiseaseDataset(test_images, test_labels, transform = ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


In [ ]:
# calculate accuraccy
def calc_accuracy(data_loader, model, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in data_loader:
            data = data.to(device=device)
            labels = labels.to(device=device)

            outputs = model(data.float())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return len(data_loader.dataset), correct, total

    # print( "Accuracy of the %d test images is %d %%"
    #     % (len(data_loader.dataset), 100 * correct / total) )

In [ ]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M",],
    "VGG19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M",],
}


class VGG_net(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG16"])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

In [ ]:
model = VGG_net(in_channels=3, num_classes=38).float()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
n_total_steps = len(train_loader)

PATH = '/content/drive/MyDrive/vgg16.pth'

path=torch.load(PATH)
model.load_state_dict(path)
model.to(DEVICE)

VGG_net(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3),

In [ ]:
# train
def train():

    for epoch in range(NUM_EPOCHS):

        # training the model
        # model.train()
        for i, (data, target) in enumerate(train_loader):
            # move tensors to GPU
            data = data.to(DEVICE)
            target = target.to(DEVICE)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float())
            # calculate the batch loss
            # target = torch.max(target, 1)[1]
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss wrt model parameters
            loss.backward()
            # perform a ingle optimization step (parameter update)
            optimizer.step()
            # update training loss
            
        # validate the model
        model.eval()
        torch.save(model.state_dict(), PATH)

        print (f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

train()

Epoch [1/25], Step [491/491], Loss: 0.8687


In [ ]:
length, correct, total = calc_accuracy(test_loader, model, DEVICE)

print( "Accuracy of the %d test images is %d %%" % (length, 100 * correct / total) )

Accuracy of the 1890 test images is 85 %
